In [1]:
import json
import random
import os
from openai import OpenAI
from pydantic import BaseModel

In [2]:
client = OpenAI(
    api_key = os.getenv('OPEN_AI_API')
)

In [3]:
def create_prompt():
    count = 0

    #day
    day_list = ['2 day','3 days', '4 days']
    day = random.choice(day_list)
    count += 1

    #price
    price_list = ["cheap budget","moderate budget","expensive budget"]
    price = random.choice(price_list)
    count += 1

    #attraction orientation
    attraction_orientation_list = ["family oriented","history oriented","activity oriented","nature oriented","food oriented","shopping oriented"]
    attraction_orientation = random.choice(attraction_orientation_list)
    count += 1

    #cuisine
    cuisine_list = ["US","Mexican","Irish","French","Italian","Greek","Indian","Chinese","Japanese","Korean","Vietnamese","Thai","Asian Fusion","Middle Eastern"]
    cuisine = random.choice(cuisine_list)
    count += 1

    #restaurant rating
    weights = [0.6, 0.3, 0.1]  # weights for 1, 2, 3, and 4 respectively
    samples = [0, 1, 2]
    coins = random.choices(samples, weights=weights)[0]
    n_list = ["good flavor","good freshness","good health","good service","good enviornment","good value"]
    restaurant_n = random.sample(n_list, coins)
    restaurant_n_str = ', '.join(restaurant_n)
    count += coins
    
    #hotel rating
    weights = [0.6, 0.3, 0.1]  # weights for 1, 2, 3, and 4 respectively
    samples = [1, 2, 3]
    coins = random.choices(samples, weights=weights)[0]
    n_list = ["good quality","good location","good service","good safety"]
    hotel_n = random.sample(n_list, coins)
    hotel_n_str = ', '.join(hotel_n)
    count += coins
    #parse the prompt
    prompt = {
        "general":day + ", " + price,
        "attraction":attraction_orientation,
        "restaurants":cuisine + (', ' if len(restaurant_n) > 0 else "") + restaurant_n_str,
        "hotel":hotel_n_str
    }

    eval_info = {
        "day":[day],
        "price":[price],
        "attraction":[attraction_orientation],
        "cuisine":[cuisine],
        "restaurants":restaurant_n,
        "hotel":hotel_n,
        "preference_count":[count]
    }

    return prompt,eval_info

In [4]:
with open('Prompts/Prompt Generation/Promp_generation_task1.txt', 'r') as file:
    system_prompt = file.read()

In [5]:
def generate_prompt_by_llm():
    #get a new prompt's information
    user_prompt, eval_info = create_prompt()
    #create the prompt with the new information
    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system","content": system_prompt,},
            {"role": "user","content": str(user_prompt),}
        ]
    )
    return chat_completion.choices[0].message.content, eval_info

In [6]:
for i in range(1):
    prompt, eval_info = generate_prompt_by_llm()
    with open(f'Prompts/Task1/Task1_prompt_{i + 2}.txt','w') as file:
        file.write(prompt)

    with open(f'Prompts/Task1_eval/Task1_eval_{i + 2}.json','w') as file:
         json.dump(eval_info, file, indent=4)